In [91]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class Net(nn.Module):
    def __init__(self):
        #下式等价于nn.Module.__init__(self)
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, 10)
        
    def forward(self, x):
        #卷积=》激活=》池化
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
      #  x = self.conv1(x)
      #  x = F.relu(x)
      #  x = F.max_pool2d(x, (2,2))
      #  x = self.conv2(x)
      #  x = F.relu(x, 2)
      #  x = F.max_pool2d(x, (2,2))
        #reshape,'-1'表示自适应
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=10, bias=True)
)

In [92]:
params = list(net.parameters())
print(len(params))

10


In [93]:
for name, parameters in net.named_parameters():
    print(name,':',parameters.size())

conv1.weight : torch.Size([6, 1, 5, 5])
conv1.bias : torch.Size([6])
conv2.weight : torch.Size([16, 6, 5, 5])
conv2.bias : torch.Size([16])
fc1.weight : torch.Size([120, 400])
fc1.bias : torch.Size([120])
fc2.weight : torch.Size([60, 120])
fc2.bias : torch.Size([60])
fc3.weight : torch.Size([10, 60])
fc3.bias : torch.Size([10])


In [94]:
input = Variable(t.randn(1, 1, 32, 32))
out = net(input)
out.size()

torch.Size([1, 10])

In [95]:
net.zero_grad()    #所有参数的梯度清零
out.backward(Variable(t.ones(1, 10)))    #反向传播

# 损失函数

In [96]:
output = net(input)
target = t.randn(10)
#target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output, target)
loss

tensor(0.8956, grad_fn=<MseLossBackward>)

# 计算图流程
# input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#           -> view -> linear -> relu -> linear -> relu -> linear
#           -> MSELoss
#           -> loss

In [97]:
net.zero_grad()
print('反向传播之前conv1.bias的梯度')
print(net.conv1.bias.grad)
loss.backward()
print('反向传播之后conv1.bias的梯度')
print(net.conv1.bias.grad)

反向传播之前conv1.bias的梯度
tensor([0., 0., 0., 0., 0., 0.])
反向传播之后conv1.bias的梯度
tensor([-0.0154, -0.0033,  0.0137,  0.0028,  0.0214,  0.0277])


# 优化器

In [98]:
import torch.optim as optim
#新建一个优化器，指定要调整的参数和学习率
optimizer = optim.SGD(net.parameters(), lr=0.01)


#训练过程中
#先梯度清零（同net.zero_grad()）
optimizer.zero_grad()

#计算损失
output = net(input)
loss = criterion(output, target)
#反向传播
loss.backward()
#更新参数
optimizer.step()